In [ ]:
from sparta.common import pool_map
import pandas as pd

In [17]:
import numpy as np
import rasterio as rio
from functools import partial
def check_image(i, data) -> np.ndarray:
    """
    Read raster window into numpy array.
    """
    sample = data.iloc[i]
    
    
    with rio.open(sample['target']) as ds:
        row, col = ds.index(sample['xgeo'], sample['ygeo'])
        data = ds.read(
            window=rio.windows.Window(col, row, 256, 256)
            )
        if np.any(data == 0):
            return None
    return i

In [18]:
train_df = pd.read_csv('./samples/train_unet_chips_256.csv')
val_df = pd.read_csv('./samples/val_unet_chips_256.csv')

In [19]:
train_mask = pool_map(partial(check_image, data=train_df), range(len(train_df)))

100%|██████████████████████████████████████████████████████████████████████████████████████| 12800/12800 [00:15<00:00, 806.67it/s]


In [20]:
val_mask = pool_map(partial(check_image, data=val_df), range(len(val_df)))

100%|███████████████████████████████████████████████████████████████████████████████████████| 3200/3200 [00:01<00:00, 1910.48it/s]


In [25]:
train_mask = [val for val in train_mask if val is not None]
val_mask = [val for val in val_mask if val is not None]

In [21]:
np.any(np.array(train_mask) == 0)

True

In [22]:
np.any(np.array(val_mask) == 0)

True

In [23]:
len(train_df), len(val_df)

(12800, 3200)

In [26]:
train_sub = train_df.iloc[train_mask]
val_sub = val_df.iloc[val_mask]

len(train_sub), len(val_sub)

(12408, 3112)

In [ ]:
train_sub.to_csv('./samples/train_unet_chips_256.csv', index=False)
val_sub.to_csv('./samples/val_unet_chips_256.csv', index=False)